In [1]:
import pandas as pd

In [2]:
dataset_path = "train.csv"

In [3]:
df = pd.read_csv(dataset_path)
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
df.shape

(20800, 5)

In [5]:
# dropping the null values 
df = df.dropna()

In [6]:
df.shape

(18285, 5)

In [7]:
# segregating dependent and independent variable
y = df.pop("label")
X = df

In [8]:
# checking the data set is balanced or not 
y.value_counts(normalize=True)*100

0    56.663932
1    43.336068
Name: label, dtype: float64

In [9]:
X.shape, y.shape

((18285, 4), (18285,))

In [10]:
# setting the vocabulary size 
voc_size = 5000

###### One-Hot representation

In [11]:
messages = X.copy()
messages = messages["title"]

In [12]:
messages = messages.reset_index()

In [13]:
messages.head()

,index,title
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ..."
2,2,Why the Truth Might Get You Fired
3,3,15 Civilians Killed In Single US Airstrike Hav...
4,4,Iranian woman jailed for fictional unpublished...


In [14]:
import nltk
import re
from nltk.corpus import stopwords

In [15]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rang\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# Dataset preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []

for i in range(0,len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    preprocessed = []
    for words in review:
        if words not in stopwords.words("english"): 
# review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
            word = ps.stem(words)
            preprocessed.append(word)
    preprocessed = " ".join(preprocessed)
    corpus.append(preprocessed)

In [ ]:
corpus

In [ ]:
from tensorflow import keras
from keras.preprocessing.text import one_hot

ohe_repr = []
for words in corpus:
    ohe_repr.append(one_hot(words,voc_size))

In [ ]:
ohe_repr

In [ ]:
# creating embeddings
from keras.preprocessing.sequence import pad_sequences

sent_len = 20
embd_docs = pad_sequences(ohe_repr,
                          padding="pre",
                          maxlen=sent_len)
print(embd_docs[0])

In [ ]:
# creating model
# importing libraries
from keras import Sequential
from keras.layers import Embedding
from keras.layers import LSTM, Dense, Bidirectional, Dropout

embd_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size, embd_vector_features, input_length=sent_len))
model.add(LSTM(100))# 100 layers
model.add(Dense(1,activation="sigmoid"))# output layer

model.compile(loss="binary_crossetrpy",optimizer="adam", metrics=["accuracy"])
print(model.summary())

In [ ]:
# creating another model
## Creating model
embedding_vector_features=40
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_len))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dropout(0.3))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())

In [ ]:
len(embd_docs), y.shape

In [ ]:
import numpy as np
X_final = np.array(embd_docs)
y_final = np.array(y)

In [ ]:
X_final.shape,y_final.shape

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, 
                                                    test_size=0.33,
                                                   random_state=42)

In [ ]:
# Model training 
model1.fit(X_train,y_train, validation_data=(X_test, y_test),
          epochs=10, batch_size=64)

In [ ]:
# performance of model
y_pred1 = model1.predict_classes(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score

In [ ]:
y_pred1

In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score
print(confusion_matrix(y_test, y_pred1))
print(accuracy_score(y_test,y_pred1))

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred1))